# In this notebook visualization for US traffic between two airports (markets) is shown for different time periods : yearly and quarterly.
# The heatmaps are interactive and markets with traffic greater than chosen percentile is displayed.

In [1]:
import pandas as pd
from bokeh.io import output_notebook,push_notebook
from bokeh.models import ( 
    ColumnDataSource,
    HoverTool,
    LinearColorMapper,
    BasicTicker,
    PrintfTickFormatter,
    ColorBar,
    FactorRange
)
from bokeh.plotting import figure,show
from bokeh.palettes import BuPu
import holoviews as hv
import numpy as np
import matplotlib.cm as cm
import matplotlib as mpl
from ipywidgets import interact
from IPython.core.display import display, HTML
output_notebook()

Loading BokehJS ...

In [2]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="ViewCode."></form>''')

In [3]:
db1b_q1=pd.read_csv('db1b_q1.csv')
db1b_q2=pd.read_csv('db1b_q2.csv')
db1b_q3=pd.read_csv('db1b_q3.csv')
db1b_q4=pd.read_csv('db1b_q4.csv')

In [4]:
def calculate_columns(*args):
    """
    Receives a number of dataframe for different quarter.
    Returns a combined dataframe with  actual number of passengers. 
    """
    list_df=list(args)
    final_df=pd.DataFrame()
    for i in range(0,len(list_df)):
        list_df[i]["ACTUAL_PASSENGERS"]=list_df[i]["PASSENGERS"]*10
        #list_df[i]["QUARTER_Actual"]=i+1
        list_df[i]=list_df[i].groupby(["ORIGIN","DEST","QUARTER"]).agg({"PASSENGERS":"sum"}).reset_index()
        final_df=final_df.append(list_df[i])
    return final_df

In [5]:
quarter_all=calculate_columns(db1b_q1,db1b_q2,db1b_q3,db1b_q4)

# Different types of data grouping

## 1. Traffic between two airports for the entire year of 2017

In [6]:
traffic_2017= quarter_all.groupby(["ORIGIN","DEST"]).agg({'PASSENGERS':sum}).reset_index()

#### Generate heat map for traffic greater than different percentile value.

In [7]:
origin=list(traffic_2017.ORIGIN.unique())
dest=list(traffic_2017.DEST.unique())
hover = HoverTool(tooltips=[
        ("ORIGIN","@ORIGIN"),
        ("DEST","@DEST"),
        ("PASSENGERS","@PASSENGERS")    
])
source= ColumnDataSource(traffic_2017)
passenger_min = traffic_2017.PASSENGERS.min()
passenger_max = traffic_2017.PASSENGERS.max()

In [8]:
colormap = cm.get_cmap("BuPu")
bokehpalette = [mpl.colors.rgb2hex(m)
                for m in colormap(np.arange(colormap.N))]

color_mapper=LinearColorMapper(
        palette=bokehpalette,low=passenger_min,high=passenger_max)

z = figure(title="US traffic density between airports", x_range=origin, y_range=dest, toolbar_location='below',
           toolbar_sticky=False, tools=[hover, 'pan', 'wheel_zoom', 'box_zoom', 'reset'], plot_width=1000, plot_height=1000)
z.rect(x='ORIGIN', y='DEST', width=1, height=1, source=source, fill_color={
    'field': 'PASSENGERS', 'transform': color_mapper}, line_color=None)


color_bar = ColorBar(color_mapper=color_mapper, major_label_text_font_size="7pt",
        ticker=BasicTicker(desired_num_ticks=8), label_standoff=6, border_line_color=None, location=(0, 0))
z.add_layout(color_bar, 'right')
z.xaxis.axis_label = 'Origin'
z.yaxis.axis_label = 'Destination'
z.xaxis.major_label_orientation=1.6

In [9]:
def update(percentile):
    selected_df = traffic_2017.loc[traffic_2017['PASSENGERS']
                                       >= traffic_2017.PASSENGERS.quantile(percentile)]
    passenger_min = selected_df.PASSENGERS.min()
    passenger_max = selected_df.PASSENGERS.max()
    origin = list(selected_df.ORIGIN.unique())
    dest = list(selected_df.DEST.unique())
    source.data = dict(
        ORIGIN=selected_df["ORIGIN"],
        DEST=selected_df["DEST"],
        PASSENGERS=selected_df["PASSENGERS"]
    )
    z.x_range.factors = origin
    z.y_range.factors = dest
    color_mapper.low = passenger_min
    color_mapper.high = passenger_max
    push_notebook(handle=heatmap_one)



In [10]:
heatmap_one=show(z, notebook_handle=True)

In [11]:
interact(update, percentile=(0.5,0.95,0.05))

interactive(children=(FloatSlider(value=0.7, description='percentile', max=0.95, min=0.5, step=0.05), Output()…

<function __main__.update(percentile)>

## 2. Traffic between two airports for  All Quarters in 2017. 

In [12]:
origin=list(quarter_all.ORIGIN.unique())
dest=list(quarter_all.DEST.unique())
hover = HoverTool(tooltips=[
        ("ORIGIN","@ORIGIN"),
        ("DEST","@DEST"),
        ("PASSENGERS","@PASSENGERS")    
])
source= ColumnDataSource(quarter_all)
passenger_min = quarter_all.PASSENGERS.min()
passenger_max = quarter_all.PASSENGERS.max()

In [13]:
colormap = cm.get_cmap("BuPu")
bokehpalette = [mpl.colors.rgb2hex(m)
                for m in colormap(np.arange(colormap.N))]

color_mapper=LinearColorMapper(
        palette=bokehpalette,low=passenger_min,high=passenger_max)

plot = figure(title="US traffic density between airports", x_range=origin, y_range=dest, toolbar_location='below',
           toolbar_sticky=False, tools=[hover, 'pan', 'wheel_zoom', 'box_zoom', 'reset'], plot_width=1000, plot_height=1000)
plot.rect(x='ORIGIN', y='DEST', width=1, height=1, source=source, fill_color={
    'field': 'PASSENGERS', 'transform': color_mapper}, line_color=None)


color_bar = ColorBar(color_mapper=color_mapper, major_label_text_font_size="7pt",
        ticker=BasicTicker(desired_num_ticks=8), label_standoff=6, border_line_color=None, location=(0, 0))
plot.add_layout(color_bar, 'right')
plot.xaxis.axis_label = 'Origin'
plot.yaxis.axis_label = 'Destination'
plot.xaxis.major_label_orientation=1.6

In [14]:
def update_all(quarter,quarterpercentile):
    selected_quarter=quarter_all.loc[quarter_all["QUARTER"]== quarter]
    selected_df = selected_quarter.loc[selected_quarter['PASSENGERS']
                                       >= selected_quarter.PASSENGERS.quantile(quarterpercentile)]
    passenger_min = selected_df.PASSENGERS.min()
    passenger_max = selected_df.PASSENGERS.max()
    origin = list(selected_df.ORIGIN.unique())
    dest = list(selected_df.DEST.unique())
    source.data = dict(
        ORIGIN=selected_df["ORIGIN"],
        DEST=selected_df["DEST"],
        PASSENGERS=selected_df["PASSENGERS"]
    )
    plot.x_range.factors = origin
    plot.y_range.factors = dest
    color_mapper.low = passenger_min
    color_mapper.high = passenger_max
    push_notebook(handle=heatmap_all)

In [15]:
heatmap_all=show(plot, notebook_handle=True)
display(HTML('<h1><a href="https://twitter.com/atrimahapatra?ref_src=twsrc%5Etfw" class="twitter-follow-button" data-show-count="false">Follow @atrimahapatra</a><script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script></h1>'))

In [16]:
interact(update_all,quarter=(1,4,1) , quarterpercentile=(0.5,0.95,0.05))

interactive(children=(IntSlider(value=2, description='quarter', max=4, min=1), FloatSlider(value=0.7, descript…

<function __main__.update_all(quarter, quarterpercentile)>